In [3]:
import numpy as np
from skimage import io
# DataSetPATH = "../input/fonts-dataset-cmp/fonts-dataset" # for kaggle
DataSetPATH = "./Dataset" # for local

In [2]:
def Preprocessing(image):
    return None